In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.2 MB/s eta 0:00:00


In [2]:
from Crypto.Cipher import AES

#3DES 암호키와 초기화 벡터 생성에 활용
from Crypto.Hash import SHA256 as SHA

from os import path

In [3]:
KSIZE = 1024

In [7]:
class myDES():

  #3DES 객체 생성시 사용할 키와 초기화 벡터 구함
  def __init__(self, keytext, ivtext):
    '''
    kettext : 3DES 암호키 생성을 위한 문자열. 16바이트 길이.
    ivtext : 초기화 벡터 위한 문자열
    '''
    hash = SHA.new() #SHA256객체 만들고 Hash 할당
    hash.update(keytext.encode('utf-8')) #주의 SHA256.update()는 유니코드 문자열을 인자로 받지 않음. python3에서 모든 문자열은 유니코드임.
    key = hash.digest() #해시 값을 추출하여 변수 key에 할당(SHA256은 256비트 해시 생성하므로, key는 256비트(32바이트)이다.)
    self.key = key[:16]

    # 블록 암호 CBC 모드로 암호하기 위해 초기화 벡터가 필요하다.
    # 3DES는 64비트 암호화 블록 크기를 가지므로, 64비트(8바이트) 초기화 벡터가 필요하다.
    hash.update(ivtext.encode('utf-8')) # 초기화 벡터를 위한 해시를 갱신
    iv = hash.digest() # 해시를 얻은 후 변수 iv에 할당
    self.iv = iv[:16] #8바이트를 슬라이싱하여 초기화 벡터값으로 할당


  # filename으로 지정된 파일 크기를 구하고, 파일 크기가 8바이트의 배수가 아니면 '0'문자열 구성
  def makeEncInfo(self, filename):
    fillersize = 0
    filesize = path.getsize(filename)
    if filesize%16 !=0:
      fillersize = 16-filesize%16

    #추가할 문자'0'개수에 대해 헤더를 만들고, 헤더와 추가할 '0'문자열을 리턴
    filler = '0'*fillersize
    header = '%d' %(fillersize)
    gap = 16-len(header)
    header += '#'*gap

    return header, filler

  #암호화
  def enc(self, filename):
    encfilename = filename + '.enc'
    header, filler = self.makeEncInfo(filename)
    aes = AES.new(self.key, AES.MODE_CBC, self.iv) # 3DES객체 des3 생성 : 인자는 암호키, 운영모드, 초기화 벡터  (ECB, CTR 모드는 초기화 벡터가 필요없다.)

    h = open(filename, 'rb')
    hh = open(encfilename, 'wb+')

    #KSIZE 만큼 읽어서 암호화 > 저장 > 파일 다 읽을 때까지 반복
    enc = header.encode('utf-8')
    content = h.read(KSIZE)
    content = enc + content
    while content:
      if len(content) < KSIZE:
        content += filler.encode('utf-8')

      enc = aes.encrypt(content)
      hh.write(enc)
      content = h.read(KSIZE)
    h.close()
    hh.close()

  #복호화
  def dec(self, encfilename):
    filename = encfilename+'.dec'
    aes = AES.new(self.key, AES.MODE_CBC, self.iv)

    h = open(filename, 'wb+')
    hh = open(encfilename, 'rb')

    #암호 파일에서 첫 8바이트를 읽어 복호. 헤더로 '#'구분자로 헤더 분리 후 첫번째 멤버를 정수로 변환
    content = hh.read(16)
    dec = aes.decrypt(content)
    header = dec.decode()
    fillersize = int(header.split('#')[0])

    content = hh.read(KSIZE)
    while content:
      dec = aes.decrypt(content)
      if len(dec) < KSIZE:
        if fillersize !=0:
          dec = dec[:-fillersize]
      h.write(dec)
      content = hh.read(KSIZE)

    h.close()
    hh.close()


def main():
  keytext = 'samsjang'
  ivtext = '1234'
  filename = 'plain.txt' # 암호화할 메시지.
  encfilename = filename + '.enc'

  myCipher = myDES(keytext, ivtext)
  ciphered = myCipher.enc(filename)
  deciphered = myCipher.dec(encfilename)

if __name__=='__main__':
  main()